In [263]:
import sklearn as sk
import spacy
nlp = spacy.blank("en")
import nltk
import re
import pandas as pd
import numpy as np
from collections import defaultdict as dd
from sklearn.feature_extraction.text import CountVectorizer
from processing_functions import Preprocess as process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
STOP = stopwords.words('english')
from textblob import TextBlob
from textblob import Word

CATEGORIES = ["14-16", "24-26", "34-36", "44-46","?"]
CATEGORIES_INDEX = [0,1,2,3,4]

In [555]:
def read(file):
    df = pd.read_csv("./COMP30027_2018S1_proj2-data/"+file+".csv",header=None)
    return df
def categorise_targets(num):
    if num <= 16:
        return 0
    if (num>=24) and (num<=26):
        return 1
    if (num>=34) and (num<=36):
        return 2
    if (num>=44) and (num<=46):
        return 3
    else:
        return 4
def tokenise(text):

    text = ' '.join(text.split())
    text = text.split(" ")
    
  
    
    return text

In [265]:
def read_and_output_train_test_split(train, test):
    
    train_raw = read(train)
    test_raw = read(test)
    
    train_raw = train_raw.loc[:, [0,6,2]]
    
    train_raw.columns = ["id","document", "age"]
    
    
    test_raw = test_raw.loc[:, [0,6,2]]
    
    test_raw.columns = ["id","document", "age"]
    
#     test_clean = remove_empty(test_raw)
    
    #train_raw = train_raw.sample(frac=1).reset_index(drop=True)
    
    
    
    train_text  = train_raw['document']
    test_text =  test_raw['document']
    train_target = train_raw['age']
    test_target = test_raw['age']
    
    train_target = train_target.apply(categorise_targets)
    test_target = test_target.apply(categorise_targets)
    
    return ((train_text, train_target),(test_text, test_target),(train_raw, test_raw))

In [266]:
data = read_and_output_train_test_split("train_raw", "dev_raw")

In [267]:
train_text = data[0][0]
train_target = data[0][1]

test_text = data[1][0]
test_target = data[1][1]


training_data = data[2][0]
test_data = data[2][1]

In [268]:
 def create_author_dict(df):
        
    author_dict = {}
    
    for index in df.index:
        try:
            author_dict[df['id'][index]]+=df['document'][index]

        except KeyError:
            author_dict.update({df['id'][index]:df['document'][index]})
            
    return author_dict

In [269]:
def target_to_unique_dict(df):
    # unique list of authors and their respective ages in index form (0->4)
    train_target_author_dict = {}
    for index in df['id'].index:
        if (df['id'][index]) not in list(train_target_author_dict.keys()):
            train_target_author_dict[df['id'][index]] = df['age'][index]

    train_target_author_series = pd.Series(train_target_author_dict)

    train_target_author_series=train_target_author_series.apply(categorise_targets)
    return train_target_author_series


In [270]:
train_target_compiled = target_to_unique_dict(training_data)

In [271]:
author_dict = create_author_dict(training_data)


In [272]:
train_text_compiled = pd.Series(list(author_dict.values()))

In [273]:
train_text_compiled

0                    Douglas Rushkoff's Frontline docu...
1              I received a short E-mail from the loca...
2                  I need more time in the day where i...
3               Entry originally posted on 8 August 20...
4           BETRAYED   2:19 am birthed the Aries moon ...
5                  Well, I decided to start this site ...
6                  My possible classes for the Spring/...
7                    06.02.00 'Why do all these people...
8              Greetings, droogies.  After a tantalisi...
9                    After Marissa's 2nd bday, my mom ...
10                 New Year's a coming The snow is fal...
11                   Well, no new art art persay, but ...
12             Buying flat. Getting new kitchen in. Fl...
13                     We went to Hagenbeck's Tierpark...
14                         Pecking Peggy             O...
15                        coffee count: XX    singing ...
16         Yep, it's officially official.  I work for ...
17            

In [578]:
from sklearn.linear_model import Perceptron
from sklearn import tree

def train(train_text_compiled, train_target_compiled):
    
    # buliding a pipeline
    text_clf = Pipeline([
        
        ('vectorizer', TfidfVectorizer(stop_words=STOP, max_df=0.50)),
        
        ('clf', Perceptron(random_state=100, warm_start=True, max_iter=2000))
        

    ])

    text_clf.fit(train_text_compiled, train_target_compiled);
    
    
      # return (("Accuracy: "+ str(100*np.mean(predicted==test_target))+"%"), predicted)
    return (text_clf)

In [579]:
text_clf = train(train_text_compiled, train_target_compiled)

In [580]:
text_clf.steps

[('vectorizer',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=0.5, max_features=None, min_df=1,
          ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',..."wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", 'urlLink', 'urlLink', 'urlLink'],
          strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('clf',
  Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
        max_iter=2000, n_iter=None, n_jo

In [581]:
test_author_dict = create_author_dict(test_data)
test_text_compiled = pd.Series(list(test_author_dict.values()))

In [582]:
predicted = text_clf.predict(test_text_compiled)

In [583]:
test_target_compiled = target_to_unique_dict(test_data)

In [584]:
np.mean(test_target_compiled==predicted)

0.62057448229792922

In [586]:
sk.metrics.confusion_matrix(test_target_compiled, predicted)

array([[471,  45,   2,   0,   0],
       [ 57, 438,  25,   4,   0],
       [ 11,  59,  19,   0,   0],
       [  3,  17,   7,   1,   0],
       [111, 203,  16,   8,   0]])